In [1]:
import numpy as np 

In [2]:
ROWS = 6  # Number of rows on the board
COLUMNS = 7  # Number of columns on the board
PLAYER = 1  # Player ID
AI = 2  # AI ID
EMPTY = 0  # Represents an empty cell
WINDOW_LENGTH = 4  # Number of pieces in a row needed to win
MAX_DEPTH = 4 # Depth of the minimax algorithm

# Helper Functions
def create_board():
    """Creates a 6x7 Connect 4 board initialized with zeros."""
    return np.zeros((ROWS, COLUMNS), dtype=int)

def drop_piece(board, row, col, piece):
    """Places a piece on the board at the specified location."""
    board[row][col] = piece

def is_valid_location(board, col):
    """Checks if a column has at least one empty cell."""
    return board[ROWS-1][col] == EMPTY

def get_next_open_row(board, col):
    """Finds the next open row in the specified column."""
    for r in range(ROWS):
        if board[r][col] == EMPTY:
            return r

def print_board(board):
     for r in range(ROWS):
        for c in range(COLUMNS):
            if board[ROWS-1-r][c] == EMPTY:
                print('\u25A1', end=' ')  # White square for empty spaces
            elif board[ROWS-1-r][c] == PLAYER:
                print('\033[31m\u25A0\033[0m', end=' ')  # Red square for player 1
            else:
                print('\033[33m\u25A0\033[0m', end=' ')  # Yellow square for player 2
        print()
    

def winning_move(board, piece):
    """Checks if a player has a winning move."""
    # Check horizontal locations for win
    for c in range(COLUMNS-3):
        for r in range(ROWS):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True

    # Check vertical locations for win
    for c in range(COLUMNS):
        for r in range(ROWS-3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True

    # Check positively sloped diagonals
    for c in range(COLUMNS-3):
        for r in range(ROWS-3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True

    # Check negatively sloped diagonals
    for c in range(COLUMNS-3):
        for r in range(3, ROWS):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True

    return False

def evaluate_window(window, piece):
    """Scores a window of four cells."""
    score = 0
    opp_piece = PLAYER if piece == AI else AI

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

def score_position(board, piece):
    """Evaluates the board and returns a score for the given piece."""
    score = 0

    # Score center column
    center_array = [int(i) for i in list(board[:, COLUMNS//2])]
    center_count = center_array.count(piece)
    score += center_count * 3

    # Score horizontal
    for r in range(ROWS):
        row_array = [int(i) for i in list(board[r,:])]
        for c in range(COLUMNS-3):
            window = row_array[c:c+WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    # Score vertical
    for c in range(COLUMNS):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(ROWS-3):
            window = col_array[r:r+WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    # Score positively sloped diagonal
    for r in range(ROWS-3):
        for c in range(COLUMNS-3):
            window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    # Score negatively sloped diagonal
    for r in range(ROWS-3):
        for c in range(COLUMNS-3):
            window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    return score

def is_terminal_node(board):
    """Checks if the game has ended either by win or draw."""
    return winning_move(board, PLAYER) or winning_move(board, AI) or len(get_valid_locations(board)) == 0

def minimax(board, depth, alpha, beta, maximizingPlayer):
    """Implements the minimax algorithm with alpha-beta pruning."""
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI):
                return (None, 100000000000000)
            elif winning_move(board, PLAYER):
                return (None, -10000000000000)
            else:  # Game is over, no more valid moves
                return (None, 0)
        else:  # Depth is zero
            return (None, score_position(board, AI))
    if maximizingPlayer:
        value = -np.inf
        best_col = np.random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, AI)
            new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                best_col = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return best_col, value
    else:  # Minimizing player
        value = np.inf
        best_col = np.random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, PLAYER)
            new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                best_col = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return best_col, value

def get_valid_locations(board):
    """Returns a list of columns that are valid moves."""
    valid_locations = []
    for col in range(COLUMNS):
        if is_valid_location(board, col):
            valid_locations.append(col)
    return valid_locations

def pick_best_move(board, piece):
    """Picks the best move based on the current board state."""
    valid_locations = get_valid_locations(board)
    best_score = -10000
    best_col = np.random.choice(valid_locations)
    for col in valid_locations:
        row = get_next_open_row(board, col)
        temp_board = board.copy()
        drop_piece(temp_board, row, col, piece)
        score = score_position(temp_board, piece)
        if score > best_score:
            best_score = score
            best_col = col
    return best_col

# Main Game Loop
def play_game():
    board = create_board()
    game_over = False
    turn = np.random.choice([PLAYER, AI])

    print_board(board)

    while not game_over:
        # Player 1 Input
        if turn == PLAYER:
            col = int(input("Player 1 Make your Selection (1-7):"))
            if is_valid_location(board, col-1):
                row = get_next_open_row(board, col-1)
                drop_piece(board, row, col-1, PLAYER)

                if winning_move(board, PLAYER):
                    print("YOU WIN!!")
                    game_over = True
        # AI Input
        else:
            col, minimax_score = minimax(board, MAX_DEPTH, -np.inf, np.inf, True)
            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, AI)

                if winning_move(board, AI):
                    print("LOSER ☺ !!")
                    game_over = True

        print()
        print_board(board)
        print()

        turn += 1
        turn = turn % 2

        if game_over:
            break

play_game()


□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 

□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ ■ □ □ □ 


□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
■ □ □ ■ □ □ □ 


□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ ■ □ □ □ 
■ □ □ ■ □ □ □ 


□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ ■ □ □ □ 
□ □ □ ■ □ □ □ 
■ □ □ ■ □ □ □ 


□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ ■ □ □ □ 
□ □ □ ■ □ □ □ 
■ □ ■ ■ □ □ □ 


□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ ■ □ □ □ 
■ □ □ ■ □ □ □ 
■ □ ■ ■ □ □ □ 


□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ ■ □ □ □ 
■ □ □ ■ □ □ □ 
■ □ ■ ■ ■ □ □ 


□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ ■ □ □ □ 
■ □ □ ■ □ □ □ 
■ ■ ■ ■ ■ □ □ 

LOSER ☺ !!

□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ □ □ □ □ 
□ □ □ ■ □ □ □ 
■ □ □ ■ □ □ □ 
■ ■ ■ ■ ■ ■ □ 

